In [1]:
# 爬文台積電及鴻海簡史 selenium輸入抓 用dict 合併相同key變df
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
import time,bs4,requests
import pandas as pd

url='https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5'
driver_path=Service('D:\chromedriver.exe')
driver=webdriver.Chrome(service=driver_path)
driver.get(url)

company_name_list=['台積電','叡揚資訊','華碩','友達','鴻海','葡萄王','三洋電機']   #,'鴻海','華碩'
big_company_list=[]
big_name_list=[list() for n in range(len(company_name_list))]
big_content_list=[list() for n in range(len(company_name_list))]

for c in company_name_list:
    driver.find_element(By.CSS_SELECTOR,'.vector-search-box-input').send_keys(c)
    driver.find_element(By.CSS_SELECTOR,'#searchButton').click()
    time.sleep(2)
    # 因為每個公司的資訊標題不同 所以要轉成list區隔 
    name_list,content_list=[],[]
    # 每次新增到大list後要清空在放下一個公司的資訊
    name_list.clear()
    content_list.clear()
    
    obj_soup=bs4.BeautifulSoup(driver.page_source,'lxml')
    obj_info=obj_soup.find('table','infobox').tbody.find_all('tr')
    for info in obj_info:
        index=info.find('th','fn org')
        en_index=info.find('td',style='text-align:center')
        th_row=info.find('th',scope='row')
        td=info.td
        # 如果tr內有符合 公司名稱 的標籤 
        if index:    
            print(index.text.strip())
            name=index.text.strip()
        # 如果tr內有符合 公司英文名稱 的標籤
        if en_index:  
            print(en_index.text.strip())
        # 如果tr內有符合 公司資訊 的標籤
        if th_row and td:
            print(th_row.text.strip(),'-',td.text.strip())
            # 先把公司資訊加到list
            name_list.append(th_row.text.strip())  # 標題
            content_list.append(td.text.strip())   # 內容
            # 轉成{標題:內容}的dict
            company_dict=dict(zip(name_list,content_list))
    #print(company_dict)
    # 把公司名稱當keys 建立更大的dict以做區隔 {公司:{標題:內容}}
    company_dict={name:company_dict}
    # 再加到list
    big_company_list.append(company_dict)
print(big_company_list)  # 重要格式

driver.close()



台灣積體電路製造股份有限公司
Taiwan Semiconductor Manufacturing Co., Ltd.

公司類型 - 上市公司
股票代號 - 臺證所：2330（1994年9月5日上市）NYSE：TSM（1997年10月20日上市）LSE：0LCVFWB：TSFA
ISIN - US8740391003
統一編號 - 22099131  （查）
成立 - 1987年2月21日 (1987-02-21)
創辦人 - 張忠謀
代表人物 - 董事長：劉德音副董事長兼總裁：魏哲家
總部 - 臺灣 300096新竹市東區力行六路8號
產業 - 電子工業
產品 - 晶圓代工服務
營業額 - ▲新臺幣1.58兆元（2021年）[1]
稅前盈餘 - ▲新臺幣5,847億元（2020年）
稅後盈餘 - ▲新臺幣5,181億元（2020年）
總資產 - ▲新臺幣2.76兆元（2020年）
資產淨值 - 65,376,200,000 美元 (2020年)
員工人數 - ▲54,000人（2022年）
實收資本額 - 新臺幣259,303,804,580元
市值 - ▲新臺幣16.38兆元（2021年2月5日）
每股盈利 - ▲新臺幣19.97元（2020年）
每股資產淨值 - ▲新臺幣71.33元（2020年）
主要股東 - 美國花旗銀行託管台積電存託憑證專戶（21.8%）行政院國家發展基金管理會（6.38%）管理人員群體（0.36%）[2]
主要子公司 - 30家[3]
網站 - www.tsmc.com
叡揚資訊股份有限公司
Galaxy Software Services Corporation
統一編號 - 22425662  （查）
成立 - 1987年
創辦人 - 張培鏞
總部 - 中華民國（臺灣） 台北市中山區德惠街9號5樓
標語口號 - 品質與價值，承諾必實現Quality & value , we are committed
產業 - 資訊服務
員工人數 - 約700人
網站 - gss.com.tw
華碩電腦股份有限公司
ASUSTeK Computer Inc.

華碩位於台北關渡的總部
商業名稱 - 華碩電腦、華碩、ASUS
公司類型 - 上市公司
股票代號 - 臺證所：2357LSE：ASKD
ISIN - TW0002357001
統一編號 -

In [2]:
# 建立空list
value_list=[list() for n in range(len(big_company_list))]  # 大list的values 就是標題和內容資訊
df_list=[list() for n in range(len(big_company_list))]

for i,obj in enumerate(big_company_list):
    #print(obj)
    # dict.kes()或values()一定要轉list才能使用
    value_list[i]=list(obj.values())
#     print(value_list[i][0].keys())
#     print(value_list[i][0].values())

    # 橫向印出
    #df=pd.DataFrame([value_list[i][0].values()],columns=value_list[i][0].keys(),index=obj.keys())
    df_list[i]=pd.DataFrame(value_list[i][0].values(),index=value_list[i][0].keys(),columns=obj.keys())
    #print(df[i])

df_=df_list[0]   # 一開始=索引1的df
n=0
for i in range(len(df_list)):
    #print(df[i])
    print(n)
    n+=1   
    # 超過len(list)就跳出
    if n<len(df_list):      
        # 某df.merge(要合併的df ,how合併方式='inner'相同的才留著 'outer'全都留著 ,left_index某df的index要引入嗎 ,right_index要合併的df的index要引入嗎)
        df_=df_.merge(df_list[n],how='outer',left_index=True,right_index=True)   # 每次合併完的df就成為索引1的df
        print('-'*30)        
    else:
        break
df_=df_.T  # 轉換方向
df_.to_csv('Company_info_from_Wiki.csv')
df_

0
------------------------------
1
------------------------------
2
------------------------------
3
------------------------------
4
------------------------------
5
------------------------------
6


,ISIN,主要子公司,主要股東,主要部門,代表人物,公司前身,公司類型,其他名稱,創辦人,員工人數,...,產品,產業,稅前盈餘,稅後盈餘,統一編號,網站,總資產,總部,股票代號,資產淨值
台灣積體電路製造股份有限公司,US8740391003,30家[3],美國花旗銀行託管台積電存託憑證專戶（21.8%）行政院國家發展基金管理會（6.38%）管理人...,NaN,董事長：劉德音副董事長兼總裁：魏哲家,NaN,上市公司,NaN,張忠謀,"▲54,000人（2022年）",...,晶圓代工服務,電子工業,"▲新臺幣5,847億元（2020年）","▲新臺幣5,181億元（2020年）",22099131 （查）,www.tsmc.com,▲新臺幣2.76兆元（2020年）,臺灣 300096新竹市東區力行六路8號,臺證所：2330（1994年9月5日上市）NYSE：TSM（1997年10月20日上市）LS...,"65,376,200,000 美元 (2020年)"
叡揚資訊股份有限公司,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,張培鏞,約700人,...,NaN,資訊服務,NaN,NaN,22425662 （查）,gss.com.tw,NaN,中華民國（臺灣） 台北市中山區德惠街9號5樓,NaN,NaN
華碩電腦股份有限公司,TW0002357001,NaN,NaN,NaN,董事長：施崇棠副董事長：徐世昌總裁：曾鏘聲共同執行長：許先越、胡書賓,弘碩電腦,上市公司,NaN,施崇棠徐世昌[1]廖敏雄謝偉琦童子賢,"台灣約6,300人，其他約7,500人（2019年）",...,主機板顯示卡音效卡 筆記型電腦桌上型電腦平板電腦 智慧型手機穿戴式裝置顯示器光碟機 網通產品...,電子電腦硬體物聯網裝置,▲新臺幣331.2億元（2020年）,▲新臺幣265.6億元（2020年）,23638777 （查）,www.asus.com,"新臺幣3,965億元（2020年）",中華民國臺灣 112019臺北市北投區立德路15號1樓[2],臺證所：2357LSE：ASKD,"新臺幣2,146億元（2020年）"
友達光電股份有限公司,US0022551073,NaN,NaN,NaN,董事長：彭双浪 總經理：柯富仁,達碁科技,股份有限公司,NaN,NaN,"38,000人（2021年）",...,薄膜電晶體液晶顯示器（TFT-LCD）低溫多晶矽（LTPS）有機發光顯示器 （AMOLED）...,光電業,NaN,NaN,84149738 （查）,auo.com,NaN,中華民國（臺灣）新竹市東區力行二路1號 （新竹科學園區）,臺證所：2409（2000年9月8日上市）,NaN
鴻海科技集團,TW0002317005,鴻準精密工業群創光電富智康集團夏普集團貝爾金廣宇科技瑞祺電通建漢科技台揚科技捷達世軟體（深圳...,鴻橋國際投資股份有限公司,消費電子產品事業群資訊系統整合與服務產品事業群通訊網絡產品事業群數位產品事業群創新數位系統事...,董事長：劉揚偉副董事長：李傑,NaN,上市公司,NaN,郭台銘,130萬人（2015）,...,電子代工組裝,電子工業,"▲新台幣 199,038,315仟元[1]","▲新台幣 164,268,571仟元[1]",NaN,www.foxconn.com,"新台幣 2,158,380,796仟元[1]",中華民國（臺灣）新北市土城區自由街2號,臺證所：2317LSE：HHPDOTCBB：HNHPF,"新台幣 1,025,067,725仟元[1]"
葡萄王生技股份有限公司,NaN,NaN,NaN,NaN,曾盛麟,NaN,股份有限公司,葡萄王,曾水照,NaN,...,養生藥品、食品類、機能性飲料,食品業,NaN,NaN,11880517 （查）,官方網站,NaN,中華民國（臺灣）桃園市平鎮區金陵路二段402號,臺證所：1707（1982年12月20日上市）,NaN
三洋電機株式會社三洋電機株式会社,NaN,NaN,NaN,NaN,代表取締役社長：佐野精一郎,NaN,NaN,NaN,井植歲男,"連結：99,875名單獨：10,823名（2008年3月）",...,半導體、家用電器、乾電池、自動扶梯、電梯、筆記型電腦、手機等,電子機器、電器產品,NaN,NaN,NaN,www.sanyo-av.com/language.php,NaN,日本大阪府守口市京阪本通二丁目5番5號,NaN,NaN
